In [6]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem
from rdkit.Chem.Draw import rdDepictor, IPythonConsole
rdDepictor.SetPreferCoordGen(True)
IPythonConsole.molSize = (350,350)

In [7]:
# Define example chiral molecules (SMILES with stereochemistry @)
smiles_list = {
    "2-Butanol (R)"      : "C[C@H](O)CC",
    "Lactic acid (L)"    : "C[C@H](O)C(=O)O",
    "1-Phenylethanol (R)":"C[C@H](O)c1ccccc1",
    "Alanine (L)"        : "C[C@H](N)C(=O)O",
}
molecules = {name: Chem.MolFromSmiles(sm) for name, sm in smiles_list.items()}
molecules

{'2-Butanol (R)': <rdkit.Chem.rdchem.Mol at 0x7fade6b644a0>,
 'Lactic acid (L)': <rdkit.Chem.rdchem.Mol at 0x7fade6b64350>,
 '1-Phenylethanol (R)': <rdkit.Chem.rdchem.Mol at 0x7fade6b64190>,
 'Alanine (L)': <rdkit.Chem.rdchem.Mol at 0x7fade6b64510>}

In [8]:
# Compute descriptors
data = []
for name, mol in molecules.items():
    desc = {
        "Name": name,
        "MolWt": Descriptors.MolWt(mol),
        "LogP": Descriptors.MolLogP(mol),
        "TPSA": Descriptors.TPSA(mol),
        "HDonors": Descriptors.NumHDonors(mol),
        "HAcceptors": Descriptors.NumHAcceptors(mol),
        "RotBonds": Descriptors.NumRotatableBonds(mol),
        "AromRings": Descriptors.NumAromaticRings(mol),
        "StereoCtr": rdMolDescriptors.CalcNumAtomStereoCenters(mol)
    }
    # Generate a 3D conformer and compute radius of gyration
    mol3d = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol3d, randomSeed=1)
    AllChem.UFFOptimizeMolecule(mol3d)
    conf = mol3d.GetConformer()
    desc["RadiusGyr"] = rdMolDescriptors.CalcRadiusOfGyration(mol3d, confId=conf.GetId())
    data.append(desc)

df = pd.DataFrame(data)
print(df.to_string(index=False))

               Name   MolWt    LogP  TPSA  HDonors  HAcceptors  RotBonds  AromRings  StereoCtr  RadiusGyr
      2-Butanol (R)  74.123  0.7772 20.23        1           1         1          0          1   1.638437
    Lactic acid (L)  90.078 -0.5482 57.53        2           2         1          0          1   1.647185
1-Phenylethanol (R) 122.167  1.7399 20.23        1           1         1          1          1   2.143548
        Alanine (L)  89.094 -0.5818 63.32        2           2         1          0          1   1.677471


In [4]:
#!/usr/bin/env python3
"""
Example: Generate chiral-sensitive descriptors for enantiomers
using RDKit + pymolPy3 + PyDescriptorC*.
"""

import os
import sys
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors

# --- 1. Generate 3D conformers and write MOL2 files with RDKit ---

def embed_and_write_mol2(smiles: str, name: str, out_dir: str) -> str:
    """
    Embed, optimize, and write a single-conformer MOL2 file.
    Returns the path to the written MOL2.
    """
    # Create RDKit molecule and add hydrogens
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
    # Embed using ETKDG and optimize with UFF
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())           # :contentReference[oaicite:3]{index=3}
    AllChem.UFFOptimizeMolecule(mol)                      # :contentReference[oaicite:4]{index=4}
    # Convert to MOL2 block
    mol2_block = Chem.MolToMolBlock(mol)                  # :contentReference[oaicite:5]{index=5}
    # Write out
    mol2_path = os.path.join(out_dir, f"{name}.mol2")
    with open(mol2_path, 'w') as f:
        f.write(mol2_block)
    return mol2_path

# Define enantiomers
enantiomers = {
    "R_1PE": "C[C@H](O)c1ccccc1",     # R-1-Phenylethanol
    "S_1PE": "C[C@@H](O)c1ccccc1",    # S-1-Phenylethanol
}

# Prepare output directory
out_dir = "mol2_files"
os.makedirs(out_dir, exist_ok=True)

mol2_paths = {name: embed_and_write_mol2(smi, name, out_dir)
              for name, smi in enantiomers.items()}

# --- 2. Launch PyMOL headless via pymolPy3 ---



In [5]:
import pymolPy3
# 0 → no GUI, headless mode :contentReference[oaicite:6]{index=6}
pm = pymolPy3.pymolPy3(0)

# --- 3. Load PyDescriptorC* plugin and process each MOL2 ---

# Initialize plugin system and load by name
import pymol.plugins
pymol.plugins.initialize(-1)                             # no GUI init
pymol.plugins.plugin_load('PyDescriptorC*')               # 

def calculate_chiral_descriptors(mol2_path: str, csv_out: str):
    """
    Instruct PyMOL to load the MOL2, run PyDescriptorC*, and save CSV.
    """
    obj_name = os.path.splitext(os.path.basename(mol2_path))[0]
    # Load the molecule
    pm(f"load {mol2_path}, {obj_name}")                    # :contentReference[oaicite:8]{index=8}
    # Run the PyDescriptorC* calculation command (plugin-specific)
    pm(f"PyDescriptorC* calculate, object={obj_name}")      # 
    # Save the descriptor CSV (plugin emits a default name)
    default_csv = f"{obj_name}_descriptors.csv"
    os.rename(default_csv, csv_out)

# Process each enantiomer
csv_dir = "descriptors"
os.makedirs(csv_dir, exist_ok=True)
descriptor_dfs = {}
for name, mol2 in mol2_paths.items():
    csv_path = os.path.join(csv_dir, f"{name}_desc.csv")
    calculate_chiral_descriptors(mol2, csv_path)
    # Read back into pandas
    df = pd.read_csv(csv_path)
    # Optionally add some RDKit descriptors
    mol = Chem.MolFromMol2Block(open(mol2).read())
    df["MolWt"] = Descriptors.MolWt(mol)                    # :contentReference[oaicite:10]{index=10}
    df["LogP"]   = Descriptors.MolLogP(mol)                  # :contentReference[oaicite:11]{index=11}
    descriptor_dfs[name] = df

# --- 4. Example usage: inspect first few descriptor rows ---
for name, df in descriptor_dfs.items():
    print(f"\n{name} descriptors:")
    print(df.head().to_string(index=False))


PyMOL failed to start, dumping diagnostics...
pymol-bundle-2.6.2-0.json
conda build: 0 https://conda.anaconda.org/schrodinger/linux-64
pymol-web-examples-2.4-1.json
conda build: 1 https://conda.anaconda.org/schrodinger/noarch
pymol-2.6.2-py312h2dc6bc7_0.json
conda build: py312h2dc6bc7_0 https://conda.anaconda.org/schrodinger/linux-64

Operating System:
Linux-5.4.0-126-generic-x86_64-with-glibc2.27
#142~18.04.1-Ubuntu SMP Thu Sep 1 16:25:16 UTC 2022
(/home/lukas/anaconda3/envs/chem/lib/python3.12/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /home/lukas/anaconda3/envs/chem/lib/python3.12/site-packages/PyQt5/../../.././libicuuc.so.75))

Python:
3.12.11 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:09:17) [GCC 11.2.0]
prefix=/home/lukas/anaconda3/envs/chem
executable=/home/lukas/anaconda3/envs/chem/bin/python
filesystemencoding=utf-8

Qt, Python and PyMOL Environment Variables:
LANG=en_US.UTF-8
PYDEVD_USE_FRAME_EV

ImportError: /home/lukas/anaconda3/envs/chem/lib/python3.12/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /home/lukas/anaconda3/envs/chem/lib/python3.12/site-packages/pymol/../../../libvtkm_cont-1.8.so.1)

 PyMOL(TM) 2.6.2 - Incentive Product
 Copyright (C) Schrodinger, LLC
 
 This Executable Build integrates and extends Open-Source PyMOL.
